In [1]:
import polars as pl

In [3]:
df = pl.read_csv('data/boat_sales.csv', ignore_errors=True)

totalHP
str
"""NA"""
"""NA"""
"""NA"""
"""NA"""
"""NA"""
…
"""0"""
"""0"""
"""0"""


In [18]:
def get_max_state(df:pl.DataFrame) -> tuple[str, int]:
    return df.select(pl.col('state').value_counts(sort=True)).unnest('state').row(0)
    


In [19]:
count = get_max_state(df)
count

('FL', 2916)

In [110]:
def max_fuel_to_hp(df:pl.DataFrame) -> tuple[pl.DataFrame, pl.DataFrame]:
    hp_df = df.select('fuelType', 'totalHP')
    cleaned = hp_df.with_columns('totalHP').filter(
    (pl.col('totalHP') != 'NA') & (pl.col('totalHP') != '0')).cast(
        {'totalHP':pl.Float64})
    return (cleaned.group_by('fuelType').agg(pl.col('totalHP').mean()), cleaned.with_columns('totalHP').filter(
    pl.col('totalHP') > cleaned.select('totalHP').quantile(.90).item()))
    


In [111]:
avg_by_hp, top_10 = max_fuel_to_hp(df)

In [112]:
avg_by_hp

fuelType,totalHP
str,f64
"""diesel""",868.752044
"""other""",191.277049
"""electric""",21.5
"""gasoline""",295.463191
"""""",118.605124


In [79]:
top_10.sort(by='totalHP', descending=True)

fuelType,totalHP
str,f64
"""diesel""",7200.0
"""diesel""",4504.0
"""diesel""",4000.0
"""diesel""",4000.0
"""diesel""",3920.0
…,…
"""gasoline""",730.0
"""gasoline""",720.0
"""gasoline""",720.0


In [77]:
top_10.select(pl.col('fuelType').value_counts())

fuelType
struct[2]
"{"""",1}"
"{""gasoline"",251}"
"{""diesel"",396}"
"{""other"",1}"


In [101]:
df.group_by('fuel_type').agg(pl.col('totalHP').mean())

ColumnNotFoundError: unable to find column "fuel_type"; valid columns: ["", "id", "type", "boatClass", "make", "model", "year", "condition", "length_ft", "beam_ft", "dryWeight_lb", "hullMaterial", "fuelType", "numEngines", "totalHP", "maxEngineYear", "minEngineYear", "engineCategory", "price", "sellerId", "city", "state", "zip", "created_date", "created_month", "created_year"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["", "id", "type", "boatClass", ...]; PROJECT */26 COLUMNS

In [98]:

clean_prices = df.filter(pl.col('price') < df.select('price').quantile(.89).item())
old_vs_new = clean_prices.with_columns(
    pl.when(
        pl.col('year') > df.select('year').quantile(.25).item()).then(
            pl.lit('New')).otherwise(pl.lit('Old')).alias('Age'))
avg_price = old_vs_new.group_by('Age').agg(pl.col('price').mean().round(2))
old_vs_new

,id,type,boatClass,make,model,year,condition,length_ft,beam_ft,dryWeight_lb,hullMaterial,fuelType,numEngines,totalHP,maxEngineYear,minEngineYear,engineCategory,price,sellerId,city,state,zip,created_date,created_month,created_year,Age
i64,i64,str,str,str,str,i64,str,f64,str,str,str,str,i64,str,str,str,str,i64,i64,str,str,str,str,i64,i64,str
1,7252689,"""power""","""power-center""","""Aquasport""","""210 CC""",1992,"""used""",21.0,"""8.5""","""3000""","""fiberglass""","""gasoline""",1,"""150""","""NA""","""NA""","""outboard-4s""",16500,217053,"""Englewood""","""FL""","""34224""","""2019-10-16""",10,2019,"""Old"""
5,7271336,"""power""","""power-deck""","""Bayliner""","""Element 180""",2020,"""new""",18.0,"""7.42""","""2000""","""fiberglass""","""gasoline""",1,"""75""","""2019""","""2019""","""outboard-4s""",26995,220570,"""Marietta""","""OH""","""45750""","""2019-11-02""",11,2019,"""New"""
14,6920067,"""power""","""power-convertible""","""Hatteras""","""Convertible""",1971,"""used""",42.0,"""13.83""","""35400""","""fiberglass""","""diesel""",2,"""840""","""1977""","""1977""","""inboard""",59500,16876,"""Mystic""","""CT""","""06355""","""2018-11-26""",11,2018,"""Old"""
15,7034740,"""power""","""power-convertible""","""Hatteras""","""Convertible""",1979,"""used""",43.0,"""14.5""","""41000""","""fiberglass""","""diesel""",2,"""900""","""1979""","""1979""","""inboard""",79500,16876,"""Clinton""","""CT""","""06357""","""2019-03-31""",3,2019,"""Old"""
16,3515831,"""sail""","""sail-racercruiser""","""Newport""","""Mk III""",1986,"""used""",30.0,"""10.67""","""NA""","""fiberglass""","""diesel""",1,"""0""","""NA""","""NA""","""""",25500,16876,"""Mystic""","""CT""","""""","""2011-03-25""",3,2011,"""Old"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
19996,7127945,"""power""","""power-pontoon""","""Silver Wave""","""250 Play""",2013,"""used""",25.0,"""8.5""","""NA""","""aluminum""","""gasoline""",0,"""NA""","""NA""","""NA""","""""",31973,34647,"""Acworth""","""GA""","""30102""","""2019-06-25""",6,2019,"""New"""
19997,7110910,"""power""","""power-runabout""","""Rinker""","""Captiva 200 MTX""",2013,"""used""",19.33,"""8""","""2795""","""fiberglass""","""gasoline""",1,"""0""","""NA""","""NA""","""""",26995,6335,"""New Baltimore""","""MI""","""48047""","""2019-06-12""",6,2019,"""New"""
19998,6935123,"""power""","""power-bay""","""Blazer""","""2200""",2019,"""new""",22.0,"""7.67""","""NA""","""fiberglass""","""gasoline""",0,"""NA""","""NA""","""NA""","""""",39995,65602,"""Seabrook""","""TX""","""77586""","""2018-12-12""",12,2018,"""New"""


In [99]:
avg_price

Age,price
str,f64
"""New""",36317.57
"""Old""",26900.39
